In [76]:
import time 
import requests
import sys
import re
import os 

import numpy as np
import pandas as pd

    #--- Librerias para Web con Selenium 
from bs4 import BeautifulSoup
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

#--- Variables Goblales ------------
randint = np.random.randint

#--- FUNCIONES DE DATA CLEANING -------------------------

def get_price(string):
    price = re.findall(r'\d..*', str(string))
    return float(price[0])

def get_num(string):
    num = re.findall(r'[0-9]+', str(string))
    return int(num[0])


def cleaning_data():
    df = pd.read_csv('extracted_dataropa_data.csv')
    df['Price'] = df['Price'].apply(get_price)
    df['Comments'] = df['Comments'].apply(get_num)
    df['Talla Pe. [%]'] = df['Talla Pe. [%]'].apply(get_num)
    df['Talla Corr. [%]'] = df['Talla Corr. [%]'].apply(get_num)
    df['Talla Gran. [%]'] = df['Talla Gran. [%]'].apply(get_num)
    df['Image'] = df['Image'].apply(lambda x: 'https:' + x)
    df.to_csv('extracted_data/ropa_data.csv', index=False)
    return df 



#--- Funciones para el scraping
def random_keypress_generator():
    keys = [Keys.ARROW_DOWN, Keys.ARROW_LEFT, Keys.ARROW_UP, Keys.ARROW_RIGHT, Keys.DOWN, Keys.END, Keys.PAGE_UP]
    action = np.random.choice(keys)
    return action

def page_to_scrape(url_to_scrape, pages_to_scrape=2, driver=None):
    list_links = []
    url_base = 'https://us.shein.com/'
    
    print(f'Starting get products links  ---')
    for i in range(8, pages_to_scrape+1):
        
        print(f'Page -- {i}')
        
        
        link = url_to_scrape + str(i)
        driver.get(link)
        time.sleep(randint(1, 6))
        try:
            elem = driver.find_element_by_css_selector('i[class="svgicon svgicon-close"]')
            elem.click()
        except:
            print('no se pudo hacer click')
        
        time.sleep(randint(0.3, 1))
        
        ActionChains(driver).send_keys(random_keypress_generator()).perform()
        time.sleep(randint(1, 2))
        
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html')
        link_scrap = soup.select('div[class="S-product-item__name"] a')
        list_links.append([link['href'] for link in link_scrap]) 
        
        
        time.sleep(randint(1, 5))
        
        
    products_links = [url_base + product_link for list_ in list_links for product_link in list_]   
    print(f'Ends get links products ---')
    return  products_links

def navegator(link, driver):
    
   
    driver.get(link)
    
    #---------------------- Navega por la pagina ---------
    time.sleep(randint(1, 6))
    #elem = driver.find_element_by_css_selector('i[class="svgicon svgicon-close"]')
    #elem.click()
    ActionChains(driver).send_keys(random_keypress_generator()).perform()
    time.sleep(0.2)
    
    ActionChains(driver).send_keys(random_keypress_generator()).perform()
    ActionChains(driver).send_keys(random_keypress_generator()).perform()
    ActionChains(driver).send_keys(random_keypress_generator()).perform()
    time.sleep(randint(1, 6))
        
    page_source = driver.page_source
    
    return page_source


def open_df():
    
    if os.path.isfile(r'extracted_data/ropa_data.csv') == False:
        
        #--------------  crear DF para el nuevo CSV ----------------
        columns = ['Product', 'Price', 'Review', 
                   'Comments','Categoría', 'Image', 'Talla Pe. [%]',
                   'Talla Corr. [%]', 'Talla Gran. [%]', 'Link']
        data = pd.DataFrame(columns=columns )
        
    else: 
       
        data = pd.read_csv('extracted_data/ropa_data.csv')
        
    return data 

    
def data_save(dict_var, data):
    data = data.append(dict_var, ignore_index=True)
    data.to_csv('extracted_data/ropa_data.csv', index= False)
    return print('Data Save')

def products_scraping(links_products, driver=None):
   
        
    print(f'Starting scraping products links ---')
    
    for num, link  in enumerate(links_products, start=1):
        
        print(f'Scraping products links--- {num} ---')
        
        #--------- Tomar infomaricon de page_source ---------
        
        page_source = navegator(link, driver)
        soup = BeautifulSoup(page_source, 'html')
        
        name = soup.select('div[class="product-intro__head-name"]')[0].text.strip()
        price = soup.select('div[class="from"] span')[0].text.strip()
        review = soup.select('div[class="ave-rate"]')[0].text.strip()
        comments = soup.select('span[class="tab active"]')[0].text.strip()
        img_link = soup.select('div[class="swiper-slide product-intro__main-item cursor-zoom-in swiper-slide-active"] img')[0]['src']
        talla =  soup.select('div [class="fit-item"]')
        catego = soup.select('div[class="bread-crumb__inner"] div:nth-last-child(2)')[0].text.strip()
        pequena = talla[0].text
        corresp = talla[1].text
        grande = talla[2].text
        
        #-------- Guardar Información ---------------
        
        dict_var = {'Product': name, 'Price':price, 'Review':review, 
               'Comments':comments,'Categoría': catego, 'Image': img_link, 'Talla Pe. [%]':pequena,
               'Talla Corr. [%]': corresp, 'Talla Gran. [%]':grande, 'Link':link}
        
        
        data = open_df()           
        data_save(dict_var, data)
        
        time.sleep(1)
        
    print(f'Scraping products successful ------')  
    
    

def main_function(pages=1):
    
    url_to_scrape = 'https://us.shein.com/Clothing-c-2030.html?ici=uses_tab01navbar04&scici=navbar_WomenHomePage~~tab01navbar04~~4~~webLink~~~~0&srctype=category&userpath=category%3EROPA&sort=8&page='
    # --------------------Inicializar Selenium -------------------#
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument('--headless') # No va a aparecer mi driver
    
    driver = webdriver.Chrome(r'C:\Program Files\ChromeDriver\chromedriver.exe', 
                                  chrome_options=options)
    # ----------------------------------------------------#
    products_links = page_to_scrape(url_to_scrape,pages, driver)
    data_products = products_scraping(products_links, driver)
    driver.quit()
    
    
    #---- clean dat ---------------
    print('Cleaning Data')
    clean_data = cleaning_data()
    print('data cleaning successful')
    return clean_data
      





In [75]:
main_function(8)

C:\Users\bkeny\AppData\Local\Temp/ipykernel_20404/2265167179.py:188: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(r'C:\Program Files\ChromeDriver\chromedriver.exe',


Starting get products links  ---
Page -- 8
Ends get links products ---
Starting scraping products links ---
Scraping products links--- 1 ---
Data Save
Scraping products links--- 2 ---
Data Save
Scraping products links--- 3 ---
Data Save
Scraping products links--- 4 ---
Data Save
Scraping products links--- 5 ---
Data Save
Scraping products links--- 6 ---
Data Save
Scraping products links--- 7 ---
Data Save
Scraping products links--- 8 ---
Data Save
Scraping products links--- 9 ---
Data Save
Scraping products links--- 10 ---
Data Save
Scraping products links--- 11 ---
Data Save
Scraping products links--- 12 ---
Data Save
Scraping products links--- 13 ---
Data Save
Scraping products links--- 14 ---
Data Save
Scraping products links--- 15 ---
Data Save
Scraping products links--- 16 ---
Data Save
Scraping products links--- 17 ---
Data Save
Scraping products links--- 18 ---
Data Save
Scraping products links--- 19 ---
Data Save
Scraping products links--- 20 ---
Data Save
Scraping products lin

,Product,Price,Review,Comments,Categoría,Image,Talla Pe. [%],Talla Corr. [%],Talla Gran. [%],Link
0,Camiseta de hombros caídos con estampado de es...,6.0,4.9,1002,Camisetas de Mujer,https:https:https:https:https:https:https:http...,4,93,3,https://us.shein.com//Skeleton-Print-Drop-Shou...
1,Camiseta de hombros caídos con estampado de es...,6.0,4.9,1002,Camisetas de Mujer,https:https:https:https:https:https:https:http...,4,93,3,https://us.shein.com//Skeleton-Print-Drop-Shou...
2,SHEIN Shorts con parche de cintura con cordón,10.0,4.9,3371,Pantalones Cortos de Mujer,https:https:https:https:https:https:https:http...,2,95,3,https://us.shein.com//SHEIN-Drawstring-Waist-P...
3,Top corto con estampado de mármol con cordón d...,11.0,4.8,1670,Camisetas de Mujer,https:https:https:https:https:https:https:http...,8,91,1,https://us.shein.com//Marble-Print-Tie-Front-C...
4,Camiseta de hombros caídos con estampado de es...,6.0,4.9,1002,Camisetas de Mujer,https:https:https:https:https:https:https:http...,4,93,3,https://us.shein.com//Skeleton-Print-Drop-Shou...
...,...,...,...,...,...,...,...,...,...,...
519,Vestido ajustado de terciopelo de cuello sweet...,20.0,5.0,9999,Vestidos de Mujer,https://img.ltwebstatic.com/images3_pi/2020/10...,2,97,1,https://us.shein.com//Ruched-Sweetheart-Neck-V...
520,SHEIN BASICS Body unicolor,7.0,5.0,9999,Bañadores de Mujer,https://img.ltwebstatic.com/images3_pi/2020/09...,2,97,1,https://us.shein.com//SHEIN-BASICS-Solid-Form-...
521,Vestido jersey de color combinado de cuello al...,20.0,4.7,4481,Vestidos de Jersey para Mujer,https://img.ltwebstatic.com/images3_pi/2020/10...,4,92,4,https://us.shein.com//Color-Block-Rib-knit-Swe...
522,SHEIN Top halter unicolor con tiras cruzadas t...,6.0,5.0,9999,Camisetas y Tops de Tirantes de Mujer,https://img.ltwebstatic.com/images2_pi/2019/09...,3,94,3,https://us.shein.com//SHEIN-Solid-Rib-knit-Cri...
